In [19]:
pip install sportsipy

In [138]:
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', None)
# import pingouin as pg
import regex as re
import altair as alt
from altair import *
import requests
import json
import seaborn as sns
import matplotlib
from matplotlib import pyplot as plt
from sportsreference.ncaab.boxscore import Boxscore
from sportsreference.ncaab.player import AbstractPlayer
import pprint
from sportsreference.ncaab.teams import Teams
from sportsreference.ncaab.roster import Roster
from sportsreference.ncaab.roster import Player
# from sportsipy.ncaab.schedule import Schedule
from sportsreference.ncaab.schedule import Schedule
import random

In [219]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from keras.layers import Dense, Dropout
from keras.models import Model, Sequential
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import make_column_transformer
from sklearn.compose import make_column_selector
import sklearn.metrics as metrics

In [4]:
teams = Teams(2021)

In [5]:
print(dir(teams))

['__call__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_add_stats_data', '_conferences_dict', '_retrieve_all_teams', '_teams', 'dataframes']


In [6]:
teamNameLst = []
for team in teams:
    teamNameLst.append(team.name)

In [7]:
# masterDf = pd.DataFrame(columns=['arena', 'boxscore_index', 'date', 'datetime', 'game', 'location', 'opponent_abbr', 'opponent_conference', 'opponent_name', 'opponent_rank', 'overtimes', 'points_against', 'points_for', 'result', 'season_losses', 'season_wins', 'streak', 'time', 'type'])
gameLst = []
teamLst = []
teamNames = []

for team in Teams():
# for team in teamNameLst[:5]:
    # print(team)
    # print(team.abbreviation.lower())
    schedule = Schedule(team.abbreviation.lower())
    for game in schedule:
        gameLst.append(game.dataframe)
        teamLst.append(team.abbreviation.lower())
        teamNames.append(team)
        

In [243]:
gameDf = pd.concat(gameLst)
gameDf['team'] = teamLst
gameDf['margin'] = gameDf['points_for']-gameDf['points_against']
gameDf['result'] = gameDf['result'].apply(lambda x: '1' if x=='Win' else 0)
gameDf = gameDf[gameDf['opponent_conference']!='Non-DI School']

In [244]:
print(gameDf.columns)
gameDf.head(10)
# gameDf.iloc[2050:2052]

Index(['arena', 'boxscore_index', 'date', 'datetime', 'game', 'location',
       'opponent_abbr', 'opponent_conference', 'opponent_name',
       'opponent_rank', 'overtimes', 'points_against', 'points_for', 'result',
       'season_losses', 'season_wins', 'streak', 'time', 'type', 'team',
       'margin'],
      dtype='object')


,arena,boxscore_index,date,datetime,game,location,opponent_abbr,opponent_conference,opponent_name,opponent_rank,...,points_against,points_for,result,season_losses,season_wins,streak,time,type,team,margin
2021-11-09-20-utah,Jon M. Huntsman Center,2021-11-09-20-utah,"Tue, Nov 9, 2021",2021-11-09 20:00:00,1,Away,utah,Pac-12,Utah,None,...,70,56,0,1,0,L 1,8:00p,Reg,abilene-christian,-14
2021-11-12-20-texas-am,Reed Arena,2021-11-12-20-texas-am,"Fri, Nov 12, 2021",2021-11-12 20:00:00,2,Away,texas-am,SEC,Texas A&M,None,...,81,80,0,2,0,L 2,8:00p,Reg,abilene-christian,-1
2021-11-16-20-texas-arlington,College Park Center,2021-11-16-20-texas-arlington,"Tue, Nov 16, 2021",2021-11-16 20:00:00,3,Away,texas-arlington,Sun Belt,UT Arlington,None,...,71,80,1,2,1,W 1,8:00p,Reg,abilene-christian,9
2021-12-04-16-abilene-christian,Teague Special Events Center,2021-12-04-16-abilene-christian,"Sat, Dec 4, 2021",2021-12-04 16:00:00,7,Home,incarnate-word,Southland,Incarnate Word,None,...,65,98,1,2,5,W 5,4:00p,Reg,abilene-christian,33
2021-12-11-19-abilene-christian,Teague Special Events Center,2021-12-11-19-abilene-christian,"Sat, Dec 11, 2021",2021-12-11 19:00:00,8,Home,drexel,CAA,Drexel,None,...,56,73,1,2,6,W 6,7:00p,Reg,abilene-christian,17
2021-12-15-20-abilene-christian,Teague Special Events Center,2021-12-15-20-abilene-christian,"Wed, Dec 15, 2021",2021-12-15 20:30:00,9,Home,cal-state-bakersfield,Big West,Cal State Bakersfield,None,...,59,69,1,2,7,W 7,8:30p,Reg,abilene-christian,10
2021-12-22-20-abilene-christian,Teague Special Events Center,2021-12-22-20-abilene-christian,"Wed, Dec 22, 2021",2021-12-22 20:00:00,11,Home,longwood,Big South,Longwood,None,...,58,74,1,2,9,W 9,8:00p,Reg,abilene-christian,16
2021-12-30-20-utah-valley,UCCU Center,2021-12-30-20-utah-valley,"Thu, Dec 30, 2021",2021-12-30 20:00:00,12,Away,utah-valley,WAC,Utah Valley,None,...,76,80,1,2,10,W 10,8:00p,Reg,abilene-christian,4
2022-01-01-21-dixie-state,Burns Arena,2022-01-01-21-dixie-state,"Sat, Jan 1, 2022",2022-01-01 21:00:00,13,Away,dixie-state,WAC,Dixie State,None,...,50,64,1,2,11,W 11,9:00p,Reg,abilene-christian,14
2022-01-06-20-abilene-christian,Teague Special Events Center,2022-01-06-20-abilene-christian,"Thu, Jan 6, 2022",2022-01-06 20:00:00,14,Home,stephen-f-austin,WAC,Stephen F. Austin,None,...,64,58,0,3,11,L 1,8:00p,Reg,abilene-christian,-6


In [245]:
gameDf = gameDf[['team','location','opponent_abbr','result','streak','margin']] #'datetime',

In [246]:
gameDf.shape

(10902, 6)

In [247]:
gameDf = gameDf.dropna()
gameDf.shape

(10808, 6)

In [248]:
features = ['team','location','opponent_abbr','result','streak'] #'datetime',
target = 'margin'

In [249]:
x = gameDf[features]
x[['team','location','opponent_abbr','streak']] = x[['team','location','opponent_abbr','streak']].astype("category")
y = gameDf[target]

C:\ProgramData\Anaconda3\envs\NICK\lib\site-packages\pandas\core\frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [250]:
column_transform = make_column_transformer(
    (
        OneHotEncoder(sparse=False, handle_unknown='ignore'),
        make_column_selector(dtype_include='category'),
        )
    )
column_transform.fit_transform(x)

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [251]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

In [252]:
lr = LinearRegression()
lr_pipeline = make_pipeline(column_transform, lr)

In [253]:
gbm = GradientBoostingRegressor()
gbm_pipeline = make_pipeline(column_transform, gbm)

In [254]:
lr_pipeline.fit(X_train,y_train)
lr_predictions = lr_pipeline.predict(X_test)
lr_predictions

gbm_pipeline.fit(X_train,y_train)
gbm_predictions = gbm_pipeline.predict(X_test)
gbm_predictions

array([ 11.82950596,   6.97983057,   9.29495108, ...,  -8.33568814,
        11.66854058, -11.86362319])

In [255]:
lr_predictions.shape
lr_predictions.flatten()

gbm_predictions.shape
gbm_predictions.flatten()

array([ 11.82950596,   6.97983057,   9.29495108, ...,  -8.33568814,
        11.66854058, -11.86362319])

In [256]:
y_test_lst = list(y_test.values)
lr_predictions_lst = list(lr_predictions)
gbm_predictions_lst = list(gbm_predictions)

In [282]:
print('Linear regression Mean Absolute Error:', metrics.mean_absolute_error(y_test, lr_predictions))  
print('Linear regression Mean Squared Error:', metrics.mean_squared_error(y_test, lr_predictions))  
print('Linear regression Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, lr_predictions)))
print(' ')
print('Gradient boosting Mean Absolute Error:', metrics.mean_absolute_error(y_test, gbm_predictions))  
print('Gradient boosting Mean Squared Error:', metrics.mean_squared_error(y_test, gbm_predictions))  
print('Gradient boosting Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, gbm_predictions)))

Linear regression Mean Absolute Error: 4728575122.399984
Linear regression Mean Squared Error: 4.834107172119528e+22
Linear regression Root Mean Squared Error: 219866031303.59924
 
Gradient boosting Mean Absolute Error: 6.872090809367936
Gradient boosting Mean Squared Error: 88.2071873424051
Gradient boosting Root Mean Squared Error: 9.391868149756208


In [264]:
df = pd.DataFrame({'Actual': y_test.values, 'Predicted': lr_predictions.flatten()})
df = df.reset_index()
# df.sort_values(by='Predicted',ascending=False)
df=df[df['index']!=2051]
lr_melted_df = pd.melt(df, id_vars=['index'], value_vars=['Actual', 'Predicted'])

df = pd.DataFrame({'Actual': y_test.values, 'Predicted': gbm_predictions.flatten()})
df = df.reset_index()
gbm_melted_df = pd.melt(df, id_vars=['index'], value_vars=['Actual', 'Predicted'])

full_df = pd.DataFrame({'Actual': y_test.values, 'LR predicted': lr_predictions.flatten(), 'GBM predicted': gbm_predictions.flatten()})
full_df.head()

,Actual,LR predicted,GBM predicted
0,5,16.046875,11.829506
1,1,9.984375,6.979831
2,14,14.039062,9.294951
3,4,10.671875,11.668541
4,4,8.687500,11.668541


In [278]:
domain = ['Actual', 'Predicted']
range_ = ['navy', 'gray']

alt.Chart(lr_melted_df).mark_line().encode(
    x='index',
    y='value',
    # color=alt.Color('variable', scale=alt.Scale(scheme='dark2'))
    color=alt.Color('variable', scale=alt.Scale(domain=domain,range=range_))
).properties(
    width=1800,
    height=300
)

alt.Chart(...)

In [279]:
domain = ['Actual', 'Predicted']
range_ = ['navy', 'gray']

alt.Chart(gbm_melted_df).mark_line().encode(
    x='index',
    y='value',
    # color=alt.Color('variable', scale=alt.Scale(scheme='dark2'))
    color=alt.Color('variable', scale=alt.Scale(domain=domain,range=range_))
).properties(
    width=1800,
    height=300
)

alt.Chart(...)

In [280]:
testEvalDf = X_test.copy()
testEvalDf['actual_margin'] = y_test.values
testEvalDf['lr_predicted_margin'] = lr_predictions.flatten()
testEvalDf['gbm_predicted_margin'] = gbm_predictions.flatten()
testEvalDf.head()

,team,location,opponent_abbr,result,streak,actual_margin,lr_predicted_margin,gbm_predicted_margin
2021-12-06-19-morehead-state,morehead-state,Home,presbyterian,1,W 4,5,16.046875,11.829506
2022-01-22-15-presbyterian,longwood,Away,presbyterian,1,W 5,1,9.984375,6.979831
2022-03-05-16-columbia,cornell,Away,columbia,1,W 2,14,14.039062,9.294951
2021-11-15-19-presbyterian,presbyterian,Home,citadel,1,W 2,4,10.671875,11.668541
2022-03-05-22-cal-poly,cal-poly,Home,california-san-diego,1,W 2,4,8.687500,11.668541


In [ ]:
def ncaa_playerdf(player_lst):
    player_df = pd.DataFrame(columns=['assist_percentage', 'assists', 'block_percentage', 'blocks', 'box_plus_minus', 'conference', 'defensive_box_plus_minus', 'defensive_rebound_percentage', 'defensive_rebounds', 'defensive_win_shares', 'effective_field_goal_percentage',
       'field_goal_attempts', 'field_goal_percentage', 'field_goals', 'free_throw_attempt_rate', 'free_throw_attempts', 'free_throw_percentage', 'free_throws', 'games_played', 'games_started', 'height', 'minutes_played', 'offensive_box_plus_minus',
       'offensive_rebound_percentage', 'offensive_rebounds', 'offensive_win_shares', 'personal_fouls', 'player_efficiency_rating', 'player_id', 'points', 'points_produced', 'position', 'steal_percentage', 'steals', 'team_abbreviation',
       'three_point_attempt_rate', 'three_point_attempts', 'three_point_percentage', 'three_pointers', 'total_rebound_percentage', 'total_rebounds', 'true_shooting_percentage', 'turnover_percentage', 'turnovers', 'two_point_attempts', 'two_point_percentage',
       'two_pointers', 'usage_percentage', 'weight', 'win_shares', 'win_shares_per_40_minutes'])
    for player in player_lst:
        try:
            x = Player(player)
            y = x.dataframe
            player_df = player_df.append(y)
        except:
            next
    return player_df